# SageMaker Inference Recommender

## 1. Introduction

SageMaker Inference Recommender is a new capability of SageMaker that reduces the time required to get machine learning (ML) models in production by automating performance benchmarking and load testing models across SageMaker ML instances. You can use Inference Recommender to deploy your model to a real-time inference endpoint that delivers the best performance at the lowest cost. 

Get started with Inference Recommender on SageMaker in minutes while selecting an instance and get an optimized endpoint configuration in hours, eliminating weeks of manual testing and tuning time.


## 2. Setup 

Note that we are using the `conda_pytorch_p36` kernel in SageMaker Notebook Instances. This is running Python 3.6 and PyTorch 1.4.0. If you'd like to use the same setup, in the AWS Management Console, go to the Amazon SageMaker console. Choose Notebook Instances, and click create a new notebook instance. Upload the current notebook and set the kernel. You can also run this in SageMaker Studio Notebooks with the `PyTorch Python 3.6 CPU Optimized` kernel.

In the next steps, you'll import standard methods and libraries as well as set variables that will be used in this notebook. The `get_execution_role` function retrieves the AWS Identity and Access Management (IAM) role you created at the time of creating your notebook instance.

In [1]:
from sagemaker import get_execution_role, Session, image_uris
import boto3
import time

In [2]:
region = boto3.Session().region_name
role = get_execution_role()
sm_client = boto3.client("sagemaker", region_name=region)
sagemaker_session = Session()
print(region)

us-east-2


## 4. Create a model archive

SageMaker models need to be packaged in `.tar.gz` files. When your SageMaker Endpoint is provisioned, the files in the archive will be extracted and put in `/opt/ml/model/` on the Endpoint. 

In this step, there are two optional tasks to:

   (1) Download a pretrained model from PyTorch Hub pytorch/vision
   
   (2) Write your inference script (inference.py)
   
These tasks are provided as a sample reference but can and should be modified when using your own trained models with Inference Recommender. 

### Optional: Download model from PyTorch Hub

Let's download the model from PyTorch Hub. By setting the variable download_the_model=False, you can skip the download and provide your own model archive.

In [11]:
!pip3 install --quiet ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [12]:
# !pip3 install --upgrade torch torchvision

In [14]:
download_the_model = True

if download_the_model:
    import torch, tarfile

    framework_version = torch.__version__
    print(framework_version)
    model_name = "resnet18"
    # Load the model, note you have to change the torchvision library to match your pytorch version
    model = torch.hub.load("pytorch/vision:v0.8.2", model_name, pretrained=True)
    inp = torch.rand(1, 3, 256, 256)
    model_trace = torch.jit.trace(model, inp)
    # Save your model
    model_trace.save("model.pth")

1.7.1


Downloading: "https://github.com/pytorch/vision/archive/v0.8.2.zip" to /home/ec2-user/.cache/torch/hub/v0.8.2.zip


In [43]:
list(inp.shape)

[1, 3, 256, 256]

In [44]:
%%writefile inference.py

import torch
import os
import logging
import base64
import io
import json
from PIL import Image
import numpy as np
import torchvision
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
COMPILED_FILE = "compiled.pt"
UNCOMPILED_FILE = "model.pth"


def predict_fn(image, model):
    return model(image)


def input_fn(request_body, content_type):
    iobytes = io.BytesIO(request_body)
    img = Image.open(iobytes)
    preprocess = transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    input_tensor = preprocess(img)
    input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model
    return input_batch.to(device)

Overwriting inference.py


### Create a model archive

To bring your own PyTorch model, SageMaker expects a single archive file in .tar.gz format, containing a model file (\*.pth) and the script (\*.py) for inference.

In [45]:
model_archive_name = "pyt-resnet18.tar.gz"

In [46]:
!tar -czf {model_archive_name} model.pth inference.py

### Upload to S3

We now have a model archive ready. We need to upload it to S3 before we can use with Inference Recommender. We'll use the SageMaker Python SDK to handle the upload.

In [47]:
# model package tarball (model artifact + inference code)
model_url = sagemaker_session.upload_data(path=model_archive_name, key_prefix="model")
print("model uploaded to: {}".format(model_url))

model uploaded to: s3://sagemaker-us-east-2-156991241640/model/pyt-resnet18.tar.gz


## 5. Create a sample payload archive

We need to create an archive that contains individual files that Inference Recommender can send to your Endpoint. Inference Recommender will randomly sample files from this archive so make sure it contains a similar distribution of payloads you'd expect in production. Note that your inference code must be able to read in the file formats from the sample payload.

*Here we are only adding four images for the example. For your own use case(s), it's recommended to add a variety of samples that is representative of your payloads.* 

In [20]:
payload_archive_name = "payload.tar.gz"

In [ ]:
## optional: download sample images
SAMPLES_BUCKET = "sagemaker-sample-files"
PREFIX = "datasets/image/pets/"
payload_location = "./sample-payload/"

if not os.path.exists(payload_location):
    os.makedirs(payload_location)
    print("Directory ", payload_location, " Created ")
else:
    print("Directory ", payload_location, " already exists")

sagemaker_session.download_data(payload_location, SAMPLES_BUCKET, PREFIX)

### Tar the payload

In [14]:
!cd ./sample-payload/ && tar czvf ../payload.tar.gz *

boxer_dog.jpg
british_blue_shorthair_cat.jpg
english_cocker_spaniel_dog.jpg
shiba_inu_dog.jpg


### Upload to S3

Next, we'll upload the packaged payload examples (payload.tar.gz) that was created above to S3.  The S3 location will be used as input to our Inference Recommender job later in this notebook. 

In [21]:
sample_payload_url = sagemaker_session.upload_data(path=payload_archive_name, key_prefix="payload")

## 6. Register model in Model Registry

In order to use Inference Recommender, you must have a versioned model in SageMaker Model Registry.  To register a model in the Model Registry, you must have a model artifact packaged in a tarball and an inference container image.  Registering a model includes the following steps:


1) **Create Model Group:** This is a one-time task per machine learning use case. A Model Group contains one or more versions of your packaged model. 

2) **Register Model Version/Package:** This task is performed for each new packaged model version. 

### Machine learning model details

Inference Recommender uses metadata about your ML model to recommend the best instance types and endpoint configurations for deployment. You can provide as much or as little information as you'd like but the more information you provide, the better your recommendations will be.

ML Frameworks: `TENSORFLOW, PYTORCH, XGBOOST, SAGEMAKER-SCIKIT-LEARN`

ML Domains: `COMPUTER_VISION, NATURAL_LANGUAGE_PROCESSING, MACHINE_LEARNING`

Example ML Tasks: `CLASSIFICATION, REGRESSION, IMAGE_CLASSIFICATION, OBJECT_DETECTION, SEGMENTATION, FILL_MASK, TEXT_CLASSIFICATION, TEXT_GENERATION, OTHER`

In [51]:
import torch

# ML framework details
framework = "pytorch"

# Note that only the framework major and minor version is supported for Neo compilation
framework_version = ".".join(torch.__version__.split(".")[:-1])

# model name as standardized by model zoos or a similar open source model
model_name = "resnet50"

# ML model details
ml_domain = "COMPUTER_VISION"
ml_task = "IMAGE_CLASSIFICATION"

print("PyT Version", framework_version)

PyT Version 1.7


### Container image URL

If you don’t have an inference container image, you can use one of the open source AWS [Deep Learning Containers (DLCs)](https://github.com/aws/deep-learning-containers) provided by AWS to serve your ML model. The code below retrieves a DLC based on your ML framework, framework version, python version, and instance type.

In [52]:
instance_type = "ml.c5.xlarge"  # Note: you can use any CPU-based instance type here, this is just to get a CPU tagged image
dlc_uri = image_uris.retrieve(
    framework,
    region,
    version=framework_version,
    py_version="py3",
    instance_type=instance_type,
    image_scope="inference",
)
dlc_uri

'763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-inference:1.7-cpu-py3'

### Create Model Group

In [53]:
model_package_group_name = "{}-cpu-models-".format(framework) + str(round(time.time()))
model_package_group_description = "{} models".format(ml_task.lower())

model_package_group_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageGroupDescription": model_package_group_description,
}

create_model_package_group_response = sm_client.create_model_package_group(
    **model_package_group_input_dict
)
print(
    "ModelPackageGroup Arn : {}".format(create_model_package_group_response["ModelPackageGroupArn"])
)

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-2:156991241640:model-package-group/pytorch-cpu-models-1646848140


### Register Model Version/Package

In this step, you'll register your pretrained model that was packaged in the prior steps as a new version in SageMaker Model Registry.  First, you'll configure the model package/version identifying which model package group this new model should be registered within as well as identify the initial approval status. You'll also identify the domain and task for your model.  These values were set earlier in the notebook 
where `ml_domain = 'COMPUTER_VISION'` and `ml_task = 'IMAGE_CLASSIFICATION'`

*Note: ModelApprovalStatus is a configuration parameter that can be used in conjunction with SageMaker Projects to trigger automated deployment pipeline.*  

In [54]:
model_package_description = "{} {} inference recommender".format(framework, model_name)

model_approval_status = "PendingManualApproval"

create_model_package_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "Domain": ml_domain.upper(),
    "Task": ml_task.upper(),
    "SamplePayloadUrl": sample_payload_url,
    "ModelPackageDescription": model_package_description,
    "ModelApprovalStatus": model_approval_status,
}

### Set up inference specification

You'll now setup the inference specification configuration for your model version.  This contains information on how the model should be hosted.

Inference Recommender expects a single input MIME type for sending requests. Learn more about [common inference data formats on SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html). This MIME type will be sent in the Content-Type header when invoking your endpoint.

In [55]:
input_mime_types = ["application/x-image"]

If you specify a set of instance types below (i.e. non-empty list), then Inference Recommender will only support recommendations within the set of instances below. For this example, we provide a list of common instance types used for image classification algorithms.

In [56]:
supported_realtime_inference_types = ["ml.c5.xlarge", "ml.m5.large", "ml.inf1.xlarge"]

### Optional: Model optimization

[Amazon SageMaker Neo](https://aws.amazon.com/sagemaker/neo) is a capability of SageMaker that automatically optimizes your ML models for any target instance type. With Neo, you don’t need to set up third-party or framework-specific compiler software, or tune the model manually for optimizing inference performance. 

Inference Recommender compiles your model using SageMaker Neo if the `ModelInput` field is provided. To prepare the inputs for model compilation, specify the input shape for your trained model.

In [57]:
data_input_configuration = "[[1,3,256,256]]"

In [58]:
modelpackage_inference_specification = {
    "InferenceSpecification": {
        "Containers": [
            {
                "Image": dlc_uri,
                "Framework": framework.upper(),
                "FrameworkVersion": framework_version,
                "NearestModelName": model_name,
                "ModelInput": {"DataInputConfig": data_input_configuration},
            }
        ],
        "SupportedContentTypes": input_mime_types,  # required, must be non-null
        "SupportedResponseMIMETypes": [],
        "SupportedRealtimeInferenceInstanceTypes": supported_realtime_inference_types,  # optional
    }
}

# Specify the model data
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0][
    "ModelDataUrl"
] = model_url

In [59]:
create_model_package_input_dict.update(modelpackage_inference_specification)

In [60]:
create_mode_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print("ModelPackage Version ARN : {}".format(model_package_arn))

ModelPackage Version ARN : arn:aws:sagemaker:us-east-2:156991241640:model-package/pytorch-cpu-models-1646848140/1


In [61]:
sm_client.describe_model_package(ModelPackageName=model_package_arn)

{'ModelPackageGroupName': 'pytorch-cpu-models-1646848140',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-2:156991241640:model-package/pytorch-cpu-models-1646848140/1',
 'ModelPackageDescription': 'pytorch resnet50 inference recommender',
 'CreationTime': datetime.datetime(2022, 3, 9, 17, 49, 13, 220000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-inference:1.7-cpu-py3',
    'ImageDigest': 'sha256:1b6b7276ef97a34269479d73c180775b1fedd31bedaa083d406d7cce9ae633c4',
    'ModelDataUrl': 's3://sagemaker-us-east-2-156991241640/model/pyt-resnet18.tar.gz',
    'ModelInput': {'DataInputConfig': '[[1,3,256,256]]'},
    'Framework': 'PYTORCH',
    'FrameworkVersion': '1.7',
    'NearestModelName': 'resnet50'}],
  'SupportedRealtimeInferenceInstanceTypes': ['ml.c5.xlarge',
   'ml.m5.large',
   'ml.inf1.xlarge'],
  'SupportedContentTypes': ['application/x-image'],
  'SupportedResponseMIMETyp

## 5: Create a SageMaker Inference Recommender Default Job

Now with your model in Model Registry, you can kick off a 'Default' job to get instance recommendations. This only requires your `ModelPackageVersionArn` and comes back with recommendations within an hour. 

The output is a list of instance type recommendations with associated environment variables, cost, throughput and latency metrics.

In [62]:
import boto3
import uuid
from sagemaker import get_execution_role

client = boto3.client("sagemaker", region)

role = get_execution_role()
default_job = uuid.uuid1()
default_response = client.create_inference_recommendations_job(
    JobName=str(default_job),
    JobDescription="Job Description",
    JobType="Default",
    RoleArn=role,
    InputConfig={"ModelPackageVersionArn": model_package_arn},
)

print(default_response)

{'JobArn': 'arn:aws:sagemaker:us-east-2:156991241640:inference-recommendations-job/3f2397c4-9fd1-11ec-bfd5-0ac52cb5fde6', 'ResponseMetadata': {'RequestId': '31338c03-0742-45c1-a3ea-2088311cc9a5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '31338c03-0742-45c1-a3ea-2088311cc9a5', 'content-type': 'application/x-amz-json-1.1', 'content-length': '120', 'date': 'Wed, 09 Mar 2022 17:49:19 GMT'}, 'RetryAttempts': 0}}


## 8. Instance Recommendation Results

Each inference recommendation includes `InstanceType`, `InitialInstanceCount`, `EnvironmentParameters` which are tuned environment variable parameters for better performance. We also include performance and cost metrics such as `MaxInvocations`, `ModelLatency`, `CostPerHour` and `CostPerInference`. We believe these metrics will help you narrow down to a specific endpoint configuration that suits your use case. 

Example:   

If your motivation is overall price-performance with an emphasis on throughput, then you should focus on `CostPerInference` metrics  
If your motivation is a balance between latency and throughput, then you should focus on `ModelLatency` / `MaxInvocations` metrics

| Metric | Description |
| --- | --- |
| ModelLatency | The interval of time taken by a model to respond as viewed from SageMaker. This interval includes the local communication times taken to send the request and to fetch the response from the container of a model and the time taken to complete the inference in the container. <br /> Units: Milliseconds |
| MaximumInvocations | The maximum number of InvokeEndpoint requests sent to an endpoint per minute. <br /> Units: None |
| CostPerHour | The estimated cost per hour for your real-time endpoint. <br /> Units: US Dollars |
| CostPerInference | The estimated cost per inference for your real-time endpoint. <br /> Units: US Dollars |

In [63]:
import pprint
import pandas as pd

finished = False
while not finished:
    inference_recommender_job = sm_client.describe_inference_recommendations_job(
        JobName=str(default_job)
    )
    if inference_recommender_job["Status"] in ["COMPLETED", "STOPPED", "FAILED"]:
        finished = True
    else:
        print("In progress")
        time.sleep(300)

if inference_recommender_job["Status"] == "FAILED":
    print("Inference recommender job failed ")
    print("Failed Reason: {}".format(inference_recommender_job["FailureReason"]))
else:
    print("Inference recommender job completed")

In progress
In progress
In progress
In progress
In progress
Inference recommender job completed


### Detailing out the result

In [64]:
data = [
    {**x["EndpointConfiguration"], **x["ModelConfiguration"], **x["Metrics"]}
    for x in inference_recommender_job["InferenceRecommendations"]
]
df = pd.DataFrame(data)
df.drop("VariantName", inplace=True, axis=1)
pd.set_option("max_colwidth", 400)
df.head()

,EndpointName,InstanceType,InitialInstanceCount,EnvironmentParameters,CostPerHour,CostPerInference,MaxInvocations,ModelLatency
0,sm-epc-179e9f77-274b-428b-80da-c8ecd1941749,ml.m5.large,1,[],0.115,0.000004,547,423
1,sm-epc-d0c77fdf-c4f9-4693-b219-cb3f9fb3f225,ml.inf1.xlarge,1,[],0.297,0.000006,801,197
2,sm-epc-1cdd499b-304e-4e7d-80f6-6710be5d7b57,ml.c5.xlarge,1,[],0.204,0.000004,758,628


In [39]:
inference_recommender_job

{'JobName': '175a92e4-9f39-11ec-bfd5-0ac52cb5fde6',
 'JobDescription': 'Job Description',
 'JobType': 'Default',
 'JobArn': 'arn:aws:sagemaker:us-east-2:156991241640:inference-recommendations-job/175a92e4-9f39-11ec-bfd5-0ac52cb5fde6',
 'RoleArn': 'arn:aws:iam::156991241640:role/service-role/AmazonSageMaker-ExecutionRole-20210302T095973',
 'Status': 'COMPLETED',
 'CreationTime': datetime.datetime(2022, 3, 8, 23, 40, 9, 593000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 3, 9, 0, 0, 48, 675000, tzinfo=tzlocal()),
 'InputConfig': {'ModelPackageVersionArn': 'arn:aws:sagemaker:us-east-2:156991241640:model-package/pytorch-cpu-models-1646781973/1'},
 'InferenceRecommendations': [{'Metrics': {'CostPerHour': 0.11500000208616257,
    'CostPerInference': 3.582554427339346e-06,
    'MaxInvocations': 535,
    'ModelLatency': 1070},
   'EndpointConfiguration': {'EndpointName': 'sm-epc-61727a21-9c56-4b3a-ba8e-eb4e96ec667b',
    'VariantName': 'sm-epc-61727a21-9c56-4b3a-ba8e-eb4e96

In [11]:
data = [{**x["Metrics"], **x["EndpointConfiguration"]} for x in rv["InferenceRecommendations"]]
df = pd.DataFrame(data)
df.drop("VariantName", inplace=True, axis=1)
df.head()

,CostPerHour,CostPerInference,MaximumInvocations,ModelLatency,EndpointName,InstanceType,InitialInstanceCount
0,0.204,0.000005,648,263596,sm-epc-fd60edf3-4ca5-4095-8e79-36d23a6a86d6,ml.c5.xlarge,1
1,0.115,0.000003,655,826019,sm-epc-745873f4-f0b2-44be-a82a-c987365eefd9,ml.c5d.large,1
2,0.115,0.000003,570,1085446,sm-epc-cd4c1a2a-af80-4eae-a125-043d82eec7b2,ml.m5.large,1


## Step 6: Custom Load Test

With an 'Advanced' Inference Recommender job, you can provide your production requirements, select instance types, tune environment variables and perform more extensive load tests. This typically takes 2-3 hours depending on your traffic pattern and number of instance types. 

The output is a list of endpoint configuration recommendations (instance type, instance count, environment variables) with associated cost, throughput and latency metrics.

In [67]:
instance_type = "ml.c5d.xlarge"
advanced_job_name = (
    model_name + "-" + instance_type.replace(".", "-") + "-load-test-" + str(round(time.time()))
)
job_description = "PyT {} on {}".format(model_name, instance_type)

In [69]:
rv = client.create_inference_recommendations_job(
    JobName=advanced_job_name,
    JobDescription=job_description,
    JobType="Advanced",
    RoleArn=role,
    InputConfig={
        "ModelPackageVersionArn": model_package_arn,
        "JobDurationInSeconds": 7200,
        "EndpointConfigurations": [
            {
                "InstanceType": instance_type,
                "EnvironmentParameterRanges": {
                    "CategoricalParameterRanges": [{"Name": "OMP_NUM_THREADS", "Value": ["2", "4"]}]
                },
            }
        ],
        "ResourceLimit": {"MaxNumberOfTests": 2, "MaxParallelOfTests": 1},
        "TrafficPattern": {
            "TrafficType": "PHASES",
            "Phases": [{"InitialNumberOfUsers": 1, "SpawnRate": 1, "DurationInSeconds": 120}],
        },
    },
    StoppingConditions={
        "MaxInvocations": 300,
        "ModelLatencyThresholds": [{"Percentile": "P95", "ValueInMilliseconds": 100}],
    },
)

print(rv)

{'JobArn': 'arn:aws:sagemaker:us-east-2:156991241640:inference-recommendations-job/resnet50-ml-c5d-xlarge-load-test-1646853966', 'ResponseMetadata': {'RequestId': '239c5dcd-577d-4355-96b1-569f84ddf496', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '239c5dcd-577d-4355-96b1-569f84ddf496', 'content-type': 'application/x-amz-json-1.1', 'content-length': '127', 'date': 'Wed, 09 Mar 2022 19:26:24 GMT'}, 'RetryAttempts': 0}}


### 8. Custom Load Test Results

Inference Recommender does benchmarking on both the endpoint configurations and here is the result. 

Analyzing load test result,    
`OMP_NUM_THREADS` = 2 shows ~20% better throughput when compared to `OMP_NUM_THREADS` = 4   
`OMP_NUM_THREADS` = 2 shows 25% saving in inference-cost when compared to `OMP_NUM_THREADS` = 4   

In all front, `OMP_NUM_THREADS` = 2  is much better endpoint configuration than `OMP_NUM_THREADS` = 4 

In [ ]:
finished = False
while not finished:
    inference_recommender_job = sm_client.describe_inference_recommendations_job(
        JobName=advanced_job_name
    )
    if inference_recommender_job["Status"] in ["COMPLETED", "STOPPED", "FAILED"]:
        finished = True
    else:
        print("In progress")
        time.sleep(300)

if inference_recommender_job["Status"] == "FAILED":
    print("Inference recommender job failed ")
    print("Failed Reason: {}".format(inference_recommender_job["FailureReason"]))
else:
    print("Inference recommender job completed")

In progress
In progress


### Detailing out the result

In [ ]:
data = [
    {**x["EndpointConfiguration"], **x["ModelConfiguration"], **x["Metrics"]}
    for x in inference_recommender_job["InferenceRecommendations"]
]
df = pd.DataFrame(data)
df.drop("VariantName", inplace=True, axis=1)
pd.set_option("max_colwidth", 400)
df.head()